In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!mkdir -p /content/pip
!pip download --exists-action=w git+https://github.com/cvlab-epfl/disk@kaggle -d /content/pip/disk

  Cloning https://github.com/cvlab-epfl/disk (to revision kaggle) to /tmp/pip-req-build-0pv6m0ks
  Running command git clone -q https://github.com/cvlab-epfl/disk /tmp/pip-req-build-0pv6m0ks
  Running command git checkout -b kaggle --track origin/kaggle
  Switched to a new branch 'kaggle'
  Branch 'kaggle' set up to track remote branch 'kaggle' from 'origin'.
  Running command git submodule update --init --recursive -q
  Cloning https://github.com/jatentaki/unets.git (to revision no_norm_at_start) to /tmp/pip-download-3i65dy5s/unets_3870c9d371ce4c37bc7143fceb9d0916
  Running command git clone -q https://github.com/jatentaki/unets.git /tmp/pip-download-3i65dy5s/unets_3870c9d371ce4c37bc7143fceb9d0916
  Running command git checkout -b no_norm_at_start --track origin/no_norm_at_start
  Switched to a new branch 'no_norm_at_start'
  Branch 'no_norm_at_start' set up to track remote branch 'no_norm_at_start' from 'origin'.
  Cloning https://github.com/jatentaki/torch-localize.git (to revision 

In [3]:
!mkdir -p wheels
!pip wheel -w wheels git+https://github.com/jatentaki/unets.git@no_norm_at_start
!pip wheel -w wheels git+https://github.com/jatentaki/torch-localize.git@flatten
!pip wheel -w wheels git+https://github.com/jatentaki/torch-dimcheck
!pip wheel -w wheels git+https://github.com/etrulls/disk@kaggle

!mkdir -p /content/pretrained
!wget https://github.com/etrulls/disk/raw/master/depth-save.pth -O /content/pretrained/disk-depth.pth
!wget https://github.com/etrulls/disk/raw/master/epipolar-save.pth -O /content/pretrained/disk-epipolar.pth

  Cloning https://github.com/jatentaki/unets.git (to revision no_norm_at_start) to /tmp/pip-req-build-yrkqk_9n
  Running command git clone -q https://github.com/jatentaki/unets.git /tmp/pip-req-build-yrkqk_9n
  Running command git checkout -b no_norm_at_start --track origin/no_norm_at_start
  Switched to a new branch 'no_norm_at_start'
  Branch 'no_norm_at_start' set up to track remote branch 'no_norm_at_start' from 'origin'.
  Created wheel for unets: filename=unets-0.1.0-py3-none-any.whl size=5183 sha256=a79b8eb3e9fb4a2f1e6d7b6f1b581e096702e2faf6820e56ddf6e91e98f77e58
  Stored in directory: /tmp/pip-ephem-wheel-cache-2fo9cz6h/wheels/e2/df/93/bf7aa9f3689d853c6b666c5d9ba423a5cffa5c08b1bb1c3710
Successfully built unets
  Cloning https://github.com/jatentaki/torch-localize.git (to revision flatten) to /tmp/pip-req-build-i71ujiz_
  Running command git clone -q https://github.com/jatentaki/torch-localize.git /tmp/pip-req-build-i71ujiz_
  Running command git checkout -b flatten --track orig

In [4]:
!pip install -f /content/wheels --no-index torch_dimcheck
!pip install -f /content/wheels --no-index torch_localize
!pip install -f /content/wheels --no-index unets
!pip install -f /content/wheels --no-index disk

Looking in links: /content/wheels
Processing ./wheels/torch_dimcheck-0.0.1-py3-none-any.whl
Looking in links: /content/wheels
Processing ./wheels/torch_localize-0.1.0-py3-none-any.whl
Looking in links: /content/wheels
Processing ./wheels/unets-0.1.0-py3-none-any.whl
Looking in links: /content/wheels
Processing ./wheels/disk-0.1.0-py3-none-any.whl
Processing ./wheels/pydegensac-0.1.2-cp37-cp37m-manylinux1_x86_64.whl
Processing ./wheels/tf_estimator_nightly-2.8.0.dev2021122109-py2.py3-none-any.whl


In [5]:
!pip install opencv-python==4.5.5.62
!pip install opencv-contrib-python==4.5.5.62

     |████████████████████████████████| 60.4 MB 1.2 MB/s 
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 66.6 MB 132 kB/s 
  Attempting uninstall: opencv-contrib-python
    Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


In [6]:
import os
import imageio
import numpy as np
from disk import DISK, Features
import torch
import torch.nn.functional as TorchFunctional
from torch.utils.data import DataLoader
from torch_dimcheck import dimchecked
from tqdm import tqdm
import argparse
from functools import partial
import h5py
import matplotlib.pyplot as plt
from glob import glob
import csv
import cv2

if not torch.cuda.is_available():
    print('You may want to enable the GPU switch?')

You may want to enable the GPU switch?


In [7]:
dry_run = True

In [8]:
import sys
sys.path.append("/content/gdrive/MyDrive/imc/superglue-pretrained-network")

from models.matching import SuperGlue, Matching
from models.utils import (compute_pose_error, compute_epipolar_error,
                          estimate_pose, make_matching_plot,
                          error_colormap, AverageTimer, pose_auc, read_image,
                          rotate_intrinsics, rotate_pose_inplane,
                          scale_intrinsics)

# Utility

In [9]:
def FlattenMatrix(M, num_digits=8):
    '''Convenience function to write CSV files.'''
    
    return ' '.join([f'{v:.{num_digits}e}' for v in M.flatten()])


def BuildCompositeImage(im1, im2, axis=1, margin=0, background=1):
    '''Convenience function to stack two images with different sizes.'''
    
    if background != 0 and background != 1:
        background = 1
    if axis != 0 and axis != 1:
        raise RuntimeError('Axis must be 0 (vertical) or 1 (horizontal')

    h1, w1, _ = im1.shape
    h2, w2, _ = im2.shape

    if axis == 1:
        composite = np.zeros((max(h1, h2), w1 + w2 + margin, 3), dtype=np.uint8) + 255 * background
        if h1 > h2:
            voff1, voff2 = 0, (h1 - h2) // 2
        else:
            voff1, voff2 = (h2 - h1) // 2, 0
        hoff1, hoff2 = 0, w1 + margin
    else:
        composite = np.zeros((h1 + h2 + margin, max(w1, w2), 3), dtype=np.uint8) + 255 * background
        if w1 > w2:
            hoff1, hoff2 = 0, (w1 - w2) // 2
        else:
            hoff1, hoff2 = (w2 - w1) // 2, 0
        voff1, voff2 = 0, h1 + margin
    composite[voff1:voff1 + h1, hoff1:hoff1 + w1, :] = im1
    composite[voff2:voff2 + h2, hoff2:hoff2 + w2, :] = im2

    return (composite, (voff1, voff2), (hoff1, hoff2))


def DrawMatches(im1, im2, kp1, kp2, matches, axis=1, margin=0, background=0, linewidth=2):
    '''Draw keypoints and matches.'''
    
    composite, v_offset, h_offset = BuildCompositeImage(im1, im2, axis, margin, background)

    # Draw all keypoints.
    for coord_a, coord_b in zip(kp1, kp2):
        composite = cv2.drawMarker(composite, (int(coord_a[0] + h_offset[0]), int(coord_a[1] + v_offset[0])), color=(255, 0, 0), markerType=cv2.MARKER_CROSS, markerSize=5, thickness=1)
        composite = cv2.drawMarker(composite, (int(coord_b[0] + h_offset[1]), int(coord_b[1] + v_offset[1])), color=(255, 0, 0), markerType=cv2.MARKER_CROSS, markerSize=5, thickness=1)
    
    # Draw matches, and highlight keypoints used in matches.
    for idx_a, idx_b in matches:
        composite = cv2.drawMarker(composite, (int(kp1[idx_a, 0] + h_offset[0]), int(kp1[idx_a, 1] + v_offset[0])), color=(0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=12, thickness=1)
        composite = cv2.drawMarker(composite, (int(kp2[idx_b, 0] + h_offset[1]), int(kp2[idx_b, 1] + v_offset[1])), color=(0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=12, thickness=1)
        composite = cv2.line(composite,
                             tuple([int(kp1[idx_a][0] + h_offset[0]),
                                   int(kp1[idx_a][1] + v_offset[0])]),
                             tuple([int(kp2[idx_b][0] + h_offset[1]),
                                   int(kp2[idx_b][1] + v_offset[1])]), color=(0, 0, 255), thickness=1)
    
    return composite

# Data reading

In [10]:
# Read the pairs file.

src = '/content/gdrive/MyDrive/imc/input'

test_samples = []
with open(f'{src}/test.csv') as f:
    reader = csv.reader(f, delimiter=',')
    for i, row in enumerate(reader):
        # Skip header.
        if i == 0:
            continue
        test_samples += [row]

if dry_run:
    for sample in test_samples:
        print(sample)

['googleurban;1cf87530;a5a9975574c94ff9a285f58c39b53d2c-0143f47ee9e54243a1b8454f3e91621a', '1cf87530', 'a5a9975574c94ff9a285f58c39b53d2c', '0143f47ee9e54243a1b8454f3e91621a']
['googleurban;6ceaefff;39563e58b2b7411da3f06427c9ee4239-0303b05ca0cb46959eac430e4b2472ca', '6ceaefff', '39563e58b2b7411da3f06427c9ee4239', '0303b05ca0cb46959eac430e4b2472ca']
['googleurban;d91db836;81dd07fb7b9a4e01996cee637f91ca1a-0006b1337a0347f49b4e651c035dfa0e', 'd91db836', '81dd07fb7b9a4e01996cee637f91ca1a', '0006b1337a0347f49b4e651c035dfa0e']


# Config

In [90]:
parser = argparse.ArgumentParser()
parser.add_argument(
    '--height', default=None, type=int,
    help='rescaled height (px). If unspecified, image is not resized in height dimension'
)
parser.add_argument(
    '--width', default=None, type=int,
    help='rescaled width (px). If unspecified, image is not resized in width dimension'
)
parser.add_argument(
    '--image-extension', default='jpg', type=str,
    help='This script ill process all files which match `image-path/*.{--image-extension}`'
)
parser.add_argument(
    '--f16', action='store_true',
    help='Store descriptors in fp16 (half precision) format'
)
parser.add_argument('--window', type=int, default=5, help='NMS window size')
parser.add_argument(
    '--n', type=int, default=None,
    help='Maximum number of features to extract. If unspecified, the number is not limited'
)
parser.add_argument(
    '--desc-dim', type=int, default=256,
    help='descriptor dimension. Needs to match the checkpoint value.'
)
parser.add_argument(
    '--mode', choices=['nms', 'rng'], default='nms',
    help=('Whether to extract features using the non-maxima suppresion mode or '
          'through training-time grid sampling technique')
)
parser.add_argument(
     '--model_path', type=str, default='/content/pretrained/disk-depth.pth',
    help="Path to the model's .pth save file"
)

parser.add_argument('--detection-scores', action='store_true')


# Hacky copy-paste: parameters go here.
args = parser.parse_args('--n 2048 --window 9 --desc-dim 256 --height 768 --width 768 --image-extension png --detection-scores'.split())
print(args)

DEV = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
CPU = torch.device('cpu')
state_dict = torch.load(args.model_path, map_location='cpu')

# For compatibility with older model saves.
if 'extractor' in state_dict:
    weights = state_dict['extractor']
elif 'disk' in state_dict:
    weights = state_dict['disk']
else:
    raise KeyError('Incompatible weight file!')

model = DISK(window=args.window, desc_dim=128)
model.load_state_dict(weights)
model = model.to(DEV)

Namespace(desc_dim=256, detection_scores=True, f16=False, height=768, image_extension='png', mode='nms', model_path='/content/pretrained/disk-depth.pth', n=2048, width=768, window=9)


# Feature extraction

In [81]:
!rm -rf features

class Image:
    def __init__(self, bitmap: ['C', 'H', 'W'], fname: str, orig_shape=None):
        self.bitmap     = bitmap
        self.fname      = fname
        if orig_shape is None:
            self.orig_shape = self.bitmap.shape[1:]
        else:
            self.orig_shape = orig_shape

    def resize_to(self, shape):
        return Image(
            self._pad(self._interpolate(self.bitmap, shape), shape),
            self.fname,
            orig_shape=self.bitmap.shape[1:],
        )

    @dimchecked
    def to_image_coord(self, xys: [2, 'N']) -> ([2, 'N'], ['N']):
        f, _size = self._compute_interpolation_size(self.bitmap.shape[1:])
        scaled = xys / f

        h, w = self.orig_shape
        x, y = scaled

        mask = (0 <= x) & (x < w) & (0 <= y) & (y < h)

        return scaled, mask

    def _compute_interpolation_size(self, shape):
        x_factor = self.orig_shape[0] / shape[0]
        y_factor = self.orig_shape[1] / shape[1]

        f = 1 / max(x_factor, y_factor)

        if x_factor > y_factor:
            new_size = (shape[0], int(f * self.orig_shape[1]))
        else:
            new_size = (int(f * self.orig_shape[0]), shape[1])

        return f, new_size

    @dimchecked
    def _interpolate(self, image: ['C', 'H', 'W'], shape) -> ['C', 'h', 'w']:
        _f, size = self._compute_interpolation_size(shape)
        return TorchFunctional.interpolate(
            image.unsqueeze(0),
            size=size,
            mode='bilinear',
            align_corners=False,
        ).squeeze(0)

    @dimchecked
    def _pad(self, image: ['C', 'H', 'W'], shape) -> ['C', 'h', 'w']:
        x_pad = shape[0] - image.shape[1]
        y_pad = shape[1] - image.shape[2]

        if x_pad < 0 or y_pad < 0:
            raise ValueError("Attempting to pad by negative value")

        return TorchFunctional.pad(image, (0, y_pad, 0, x_pad))

    
class SceneDataset:
    def __init__(self, image_path, crop_size=(None, None)):
        self.image_path = image_path
        self.crop_size  = crop_size
        self.names = [p for p in os.listdir(image_path) \
                      if p.endswith(args.image_extension)]

    def __len__(self):
        return len(self.names)

    def __getitem__(self, ix):
        name   = self.names[ix]
        path   = os.path.join(self.image_path, name) 
        img    = np.ascontiguousarray(imageio.imread(path))
        tensor = torch.from_numpy(img).to(torch.float32)

        if len(tensor.shape) == 2: # some images may be grayscale
            tensor = tensor.unsqueeze(-1).expand(-1, -1, 3)

        bitmap              = tensor.permute(2, 0, 1) / 255.
        extensionless_fname = os.path.splitext(name)[0]
        image = Image(bitmap, extensionless_fname)

        if self.crop_size != (None, None):
            image = image.resize_to(self.crop_size)

        return image

    @staticmethod
    def collate_fn(images):
        bitmaps = torch.stack([im.bitmap for im in images], dim=0)
        
        return bitmaps, images


def extract(dataset, save_path):
    dataloader = DataLoader(
        dataset,
        batch_size=1,
        pin_memory=True,
        collate_fn=dataset.collate_fn,
        num_workers=4,
    )

    if args.mode == 'nms':
        extract = partial(
            model.features,
            kind='nms',
            window_size=args.window,
            cutoff=0.,
            n=args.n
        )
    else:
        extract = partial(model.features, kind='rng')

    os.makedirs(os.path.join(save_path), exist_ok=True)
    keypoint_h5   = h5py.File(os.path.join(save_path, 'keypoints.h5'), 'w')
    descriptor_h5 = h5py.File(os.path.join(save_path, 'descriptors.h5'), 'w')
    if args.detection_scores:
        score_h5 = h5py.File(os.path.join(save_path, 'scores.h5'), 'w')

    pbar = tqdm(dataloader)
    for bitmaps, images in pbar:
        bitmaps = bitmaps.to(DEV, non_blocking=True)

        with torch.no_grad():
            try:
                batched_features = extract(bitmaps)
            except RuntimeError as e:
                if 'U-Net failed' in str(e):
                    msg = ('Please use input size which is multiple of 16 (or '
                           'adjust the --height and --width flags to let this '
                           'script rescale it automatically). This is because '
                           'we internally use a U-Net with 4 downsampling '
                           'steps, each by a factor of 2, therefore 2^4=16.')

                    raise RuntimeError(msg) from e
                else:
                    raise

        for features, image in zip(batched_features.flat, images):
              features = features.to(CPU)

              kps_crop_space = features.kp.T
              kps_img_space, mask = image.to_image_coord(kps_crop_space)

              keypoints   = kps_img_space.numpy().T[mask]
              descriptors = features.desc.numpy()[mask]
              scores      = features.kp_logp.numpy()[mask]

              order = np.argsort(scores)[::-1]

              keypoints   = keypoints[order]
              descriptors = descriptors[order].T
              scores      = scores[order]

              print(descriptors.shape)

              assert descriptors.shape[0] == args.desc_dim
              assert keypoints.shape[1] == 2

              if args.f16:
                  descriptors = descriptors.astype(np.float16)

              keypoint_h5.create_dataset(image.fname, data=keypoints)
              descriptor_h5.create_dataset(image.fname, data=descriptors)

              if args.detection_scores:
                  score_h5.create_dataset(image.fname, data=scores)

              pbar.set_postfix(n=keypoints.shape[0])

In [82]:
# Extract features for every image in every folder.
for dataset_folder in glob('/content/gdrive/MyDrive/imc/input/test_images/*'):
    batch_id = dataset_folder.split('/')[-1]
    print(f'Processing "{dataset_folder}"')
    dataset = SceneDataset(dataset_folder, crop_size=(args.height, args.width))
    described_samples = extract(dataset, f'features/{batch_id}')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Processing "/content/gdrive/MyDrive/imc/input/test_images/1cf87530"


  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:749: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
 50%|█████     | 1/2 [00:12<00:12, 12.63s/it, n=1186]

(256, 1186)


100%|██████████| 2/2 [00:24<00:00, 12.16s/it, n=1107]


...

 50%|█████     | 1/2 [00:10<00:10, 10.85s/it, n=838]

(256, 838)


100%|██████████| 2/2 [00:20<00:00, 10.04s/it, n=777]

(256, 777)


100%|██████████| 2/2 [00:20<00:00, 10.26s/it, n=777]


# Matching

In [83]:
device = "cuda" if torch.cuda.is_available() else "cpu"
resize = [-1, ]
resize_float = True

config = {
    "superglue": {
        "weights": "outdoor",
        "sinkhorn_iterations": 20,
        "match_threshold": 0.2,
        "descriptor_dim" : 256
    }
}
matching = Matching(config).eval().to(device)

Loaded SuperPoint model
Loaded SuperGlue model ("outdoor" weights)


In [84]:
def totensor(frame, device):
    return torch.from_numpy(frame).float()[None, None].to(device)

def score2tensor(frame, device):
    return torch.from_numpy(frame)[None].float().to(device)

def kpts2tensor(frame, device):
    return torch.from_numpy(frame).float().to(device)

In [51]:
import cv2
import gc

how_many_to_fill = -1

F_dict = {}
for i, row in enumerate(test_samples):
    sample_id, batch_id, image_1_id, image_2_id = row

    with h5py.File(f'features/{batch_id}/keypoints.h5', 'r') as kp_dict, h5py.File(f'features/{batch_id}/descriptors.h5', 'r') as desc_dict:

        desc0, desc1 = frame2tensor(desc_dict[image_1_id][()], device), frame2tensor(desc_dict[image_2_id][()], device)
        kpts0, kpts1 = frame2tensor(kp_dict[image_1_id][()], device), frame2tensor(kp_dict[image_1_id][()], device)

        pred = matching({"image0": desc0, "image1": desc1})
        pred = {k: v[0].detach().cpu().numpy() for k, v in pred.items()}
        kpts1, kpts2 = pred["keypoints0"], pred["keypoints1"]
        matches, conf = pred["matches0"], pred["matching_scores0"]

        valid = matches > -1
        mkpts1 = kpts1[valid]
        mkpts2 = kpts2[matches[valid]]
        mconf = conf[valid]

      
        if len(mkpts1) > 8:
            F, inliers = cv2.findFundamentalMat(mkpts1, 
                                                mkpts2, 
                                                cv2.USAC_MAGSAC,
                                                ransacReprojThreshold=0.75,
                                                confidence=0.99,
                                                maxIters=10000)
        
            inliers = inliers > 0
            assert F.shape == (3, 3), 'Malformed F?'
            F_dict[sample_id] = F
        else:
            F_dict[sample_id] = np.zeros((3, 3))
            continue
        gc.collect()

        if dry_run:
            image_1 = cv2.cvtColor(cv2.imread(f'{src}/test_images/{batch_id}/{image_1_id}.png'), cv2.COLOR_BGR2RGB)
            image_2 = cv2.cvtColor(cv2.imread(f'{src}/test_images/{batch_id}/{image_2_id}.png'), cv2.COLOR_BGR2RGB)
            matches_after_ransac = np.array([match for match, is_inlier in zip(matches, inliers) if is_inlier])

            im_inliers = DrawMatches(image_1, image_2, kp_dict[image_1_id][()], kp_dict[image_2_id][()], matches_after_ransac)
            fig = plt.figure(figsize=(15, 15))
            plt.title(f'{image_1_id}-{image_2_id}')
            plt.imshow(im_inliers)
            plt.axis('off')
            plt.show()


NameError: ignored

In [85]:
with h5py.File(f'features/{batch_id}/keypoints.h5', 'r') as kp_dict, h5py.File(f'features/{batch_id}/descriptors.h5', 'r') as desc_dict, h5py.File(f'features/{batch_id}/scores.h5', 'r') as scores_dict:

        desc0, desc1 = totensor(desc_dict[image_1_id][()], device), totensor(desc_dict[image_2_id][()], device)
        kpts0, kpts1 = kpts2tensor(kp_dict[image_1_id][()], device), totensor(kp_dict[image_2_id][()], device)
        scores0, scores1 = score2tensor(scores_dict[image_1_id][()], device), score2tensor(scores_dict[image_2_id][()], device)
        
        pred = matching({"keypoints0": kpts0, "keypoints1": kpts1, "descriptors0": desc0, "descriptors1": desc1, 'image0': desc0, 'image1': desc1, 'scores0': scores0, 'scores1': scores1})

KeyError: ignored

In [69]:
print(desc0.shape)

torch.Size([1, 1, 128, 2048])


In [70]:
print(kpts0.shape)

torch.Size([2048, 2])


In [71]:
print(scores0.shape)

torch.Size([1, 2048])
